**Problem Statement:**

It happens all the time: someone gives you data containing malformed strings, Python lists and missing data. How do you tidy up so you get on with the analysis?

Take this monstrosity as the DataFrame to use in the following puzzles:

	df = pd.DataFrame({'From_To':['LoNDon_paris, 'MAdrid_miLAN', 'londON_StockhOlm',
    
		'Budapest_PaRis', 'Brussels_londON'],

		'FlightNumber': [10045, np.nan,10065, np.nan, 10085],

		'RecentDelays': [[23,47],[],[24,43,87],[13],[67,32]],

		'Airline': ['KLM(!)','<Air France> (12)', '(British Airways.)',

			'12. Air France', "Swiss Air"]})
	

In [67]:
import pandas as pd
import numpy as np
df = pd.DataFrame({
                    'From_To':['LoNDon_paris', 'MAdrid_miLAN', 'londON_StockhOlm','Budapest_PaRis', 'Brussels_londON'],
                    'FlightNumber': [10045, np.nan,10065, np.nan, 10085],'RecentDelays': [[23,47],[],[24,43,87],[13],[67,32]],
                    'Airline': ['KLM(!)','<Air France> (12)', '(British Airways.)','12. Air France', "Swiss Air"]
                  })
print(df)

            From_To  FlightNumber  RecentDelays             Airline
0      LoNDon_paris       10045.0      [23, 47]              KLM(!)
1      MAdrid_miLAN           NaN            []   <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways.)
3    Budapest_PaRis           NaN          [13]      12. Air France
4   Brussels_londON       10085.0      [67, 32]           Swiss Air


    1. Some values in the FlightNumber column are missing. These numbers are meant to increase by 10 with each row so    10055 and 10075 need to be put in place. Fill in these numbers and make the column an integer column (instead of a   float column).

In [69]:
flightnumber=10
df1 = df
for item, frame in df['FlightNumber'].iteritems():
    if pd.isnull(frame):
        a = flightnumber + 10
        df1['FlightNumber'][item] = a
    else: 
        flightnumber = frame
        

print(df1)

            From_To  FlightNumber  RecentDelays             Airline
0      LoNDon_paris       10045.0      [23, 47]              KLM(!)
1      MAdrid_miLAN       10055.0            []   <Air France> (12)
2  londON_StockhOlm       10065.0  [24, 43, 87]  (British Airways.)
3    Budapest_PaRis       10075.0          [13]      12. Air France
4   Brussels_londON       10085.0      [67, 32]           Swiss Air


	2. The From_To column would be better as two separate columns! Split each string on the underscore delimiter _ to    give a new temporary DataFrame with the correct values. Assign the correct column names to this temporary DataFrame.

In [70]:
df2 = df['From_To'].str.split('_', 1, expand=True).rename(columns={0:'From', 1:'To'})
print(df2)

       From         To
0    LoNDon      paris
1    MAdrid      miLAN
2    londON  StockhOlm
3  Budapest      PaRis
4  Brussels     londON


	3. Notice how the capitalisation of the city names is all mixed up in this temporary DataFrame. Standardise the      strings so that only the first letter is uppercase (e.g. "londON" should become "London".)

In [71]:
df4 = df2
for index, row in df2.iterrows():
    df4['From'][index] = row['From'].title()
    df4['To'][index] = row['To'].title()

print(df4)


       From         To
0    London      Paris
1    Madrid      Milan
2    London  Stockholm
3  Budapest      Paris
4  Brussels     London


	4. Delete the From_To column from df and attach the temporary DataFrame from the previous questions.

In [72]:
df3 = df.loc[:, ~df.columns.str.contains('From_To')]
df3 = df4.join(df3)
print(df3)

       From         To  FlightNumber  RecentDelays             Airline
0    London      Paris       10045.0      [23, 47]              KLM(!)
1    Madrid      Milan       10055.0            []   <Air France> (12)
2    London  Stockholm       10065.0  [24, 43, 87]  (British Airways.)
3  Budapest      Paris       10075.0          [13]      12. Air France
4  Brussels     London       10085.0      [67, 32]           Swiss Air


	5. In the RecentDelays column, the values have been entered into the DataFrame as a list. We would like each first   value in its own column, each second value in its own column, and so on. If there isn't an Nth value, the value      should be NaN.

	Expand the series of lists into a DataFrame named delays, rename the columns delay_1, delay_2, etc. and replace the  unwanted RecentDelays column in df with delays.

In [142]:
print(df3,'\n')

total_rows=len(df.axes[0])


tempsize = 0
for item, frame in df['RecentDelays'].iteritems():
    if tempsize < len(frame): 
        tempsize = len(frame)      

index = range(total_rows)
df6 =  pd.DataFrame(index = index, columns = ["delay_{}".format(i + 1) for i in range(3) ])

n=0
for item, frame in df['RecentDelays'].iteritems():
    for j in range(tempsize):
        n = j +1
        if (len(frame) != 0 and j < len(frame)) :
            df6["delay_"+str(n)][item] = frame[j]
        else :
            df6["delay_"+str(n)][item] = np.nan
        
print("After dividing RecentDelays column into delay_1, delay_2 ...")
print('-'*60)
print(df6)     

       From         To  FlightNumber  RecentDelays             Airline
0    London      Paris       10045.0      [23, 47]              KLM(!)
1    Madrid      Milan       10055.0            []   <Air France> (12)
2    London  Stockholm       10065.0  [24, 43, 87]  (British Airways.)
3  Budapest      Paris       10075.0          [13]      12. Air France
4  Brussels     London       10085.0      [67, 32]           Swiss Air 

After dividing RecentDelays column into delay_1, delay_2 ...
------------------------------------------------------------
  delay_1 delay_2 delay_3
0      23      47     NaN
1     NaN     NaN     NaN
2      24      43      87
3      13     NaN     NaN
4      67      32     NaN
